In [1]:
import pandas as pd
import numpy as np
import os, re
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']

## Get file path

In [2]:
init_path = "data\\raw"
rectifier = []
corrosion = []
for file in os.listdir(init_path):
    if "整流器檢測數據" in file:
        rectifier.append(os.path.join(init_path, file))
    elif "陰極防蝕檢測曲線" in file:
        corrosion.append(os.path.join(init_path, file))

## Merge rectifier data

In [3]:
df = pd.DataFrame()
for path in rectifier:
    print(path)
    for sheet in ["中港溪北", "後龍溪南"]:
        df1 = pd.read_excel(path, sheet_name = sheet, header = 2)

        df2 = df1.T.reset_index(drop = True)
        df2.columns = df2.iloc[0]
        df2 = df2.iloc[1:]
        df2 = df2.reset_index(drop = True)
        df2["整流器編號"] = int(re.findall(r"\d+", df2.loc[0, "整流器編號"])[0])
        df2.iloc[:, 3:13] = df2.iloc[:, 3:13].applymap(lambda X: X.replace("mv", "").replace("Ω", "").replace("A", "").replace("V", ""))
        # df2["項目"] = df2["項目"].str[-1]

        for col in ["交流輸入  A/V", "直流輸出  A/V"]:
            av = df2[col].str.split("/")
            name = col.split(" ")[0]
            df2[[f"{name}_A", f"{name}_V"]] = None
            df2[[f"{name}_A", f"{name}_V"]] = av.to_list()
            df2 = df2.drop(col, axis = 1)
        
        df2.columns = [
            '整流器', '項目', '檢測日期', '排流點電位_mv', '自動回授電位_mv', '接地棒阻抗_Ω', '鈦陽極接地電阻A1_Ω', '鈦陽極接地電阻A2_Ω', '鈦陽極接地電阻A3_Ω', 
            '鈦陽極接地電阻A4_Ω', '鈦陽極總接地電阻_Ω', '本次電力_KWH', '平均電力值_KWH/日', '交流輸入_A', '交流輸入_V', '直流輸出_A', '直流輸出_V'
            ]

        df = pd.concat([df, df2], ignore_index = True)


df = df.sort_values(["整流器", "項目"])
df = df.drop_duplicates(subset = ["整流器", "項目"], keep = "last")
df = df.reset_index(drop = True)
print(df.shape)
df.to_csv("data/parser/rectifier.csv", index = False)

data\raw\111Q1整流器檢測數據.xls
data\raw\111Q2整流器檢測數據.xls
data\raw\111Q3整流器檢測數據.xls
data\raw\111Q4整流器檢測數據.xls
(20, 17)


## Merge corrosion data

In [4]:
seasons = [["第一季", 1], ["第二季", 2], ["第三季", 3], ["第四季", 4]]
df = pd.DataFrame()
for path in corrosion:
    print(path)
    for season in seasons:
        df1 = pd.read_excel(path, sheet_name = season[0])

        df2 = df1.iloc[1:8]
        df2 = df2.T.reset_index(drop = True)

        df2.iloc[0, 0] = "檢測器"
        df2.columns = df2.iloc[0]
        df2 = df2.iloc[1:]

        df2.insert(0, "季節", season[1])

        df = pd.concat([df, df2], ignore_index = True)

print(df.shape)
df.to_csv("data/parser/corrosion.csv", index = False)

data\raw\111年陰極防蝕檢測曲線_苗栗縣.xls
(40, 8)


## Merge rectifier & corrosion

In [5]:
rectifier = pd.read_csv("data/parser/rectifier.csv")
print(rectifier.shape)
rectifier.head()

(20, 17)


,整流器,項目,檢測日期,排流點電位_mv,自動回授電位_mv,接地棒阻抗_Ω,鈦陽極接地電阻A1_Ω,鈦陽極接地電阻A2_Ω,鈦陽極接地電阻A3_Ω,鈦陽極接地電阻A4_Ω,鈦陽極總接地電阻_Ω,本次電力_KWH,平均電力值_KWH/日,交流輸入_A,交流輸入_V,直流輸出_A,直流輸出_V
0,24,109Q3,2020-07-21 00:00:00,-1447,-1348,6.32,4.56,4.52,4.65,5.91,4.28,3541,6.750000,0.65,230.9,1.55,3.80
1,24,109Q4,2020-12-01 00:00:00,-1371,-1298,6.86,5.72,5.73,5.85,6.81,5.48,4303,5.730000,1.74,231.6,1.81,1.87
2,24,110Q1,2021-01-22 00:00:00,-1492,-1318,3.20,2.28,2.33,2.35,3.64,2.22,4655,6.770000,0.68,231.0,1.56,3.77
3,24,110Q2,2021-04-29 00:00:00,-1217,-1182,6.82,5.43,5.65,6.73,7.44,5.36,5288,6.530000,0.84,235.0,2.10,3.76
4,24,110Q3,2021-07-20 00:00:00,-1457,-1198,1.10,1.79,2.61,2.84,3.19,2.19,5750,5.634146,0.74,230.2,1.75,3.77


In [6]:
corrosion = pd.read_csv("data/parser/corrosion.csv")
print(corrosion.shape)
corrosion.head()

(40, 8)


,季節,檢測器,G95汽油管線,DS柴油管線,JF航空燃油管線,G92汽油管線,最低電位標準值,最高電位標準值
0,1,T168,-1319,-1318,-1312,-1311,-1500,-850
1,1,T167,-1408,-1405,-1400,-1402,-1500,-850
2,1,T166,-1447,-1441,-1419,-1442,-1500,-850
3,1,T165,-1451,-1469,-1427,-1460,-1500,-850
4,1,T164,-1411,-1398,-1442,-1440,-1500,-850


In [7]:
rectifier["季節"] = rectifier["項目"].str[-1]
rectifier["季節"] = rectifier["季節"].astype(int)

corrosion["檢測器"] = corrosion["檢測器"].str[1:]
corrosion["檢測器"] = corrosion["檢測器"].astype(int)

idx_111 = rectifier["項目"].apply(lambda X: '111' in X)
df = rectifier[idx_111]

df

,整流器,項目,檢測日期,排流點電位_mv,自動回授電位_mv,接地棒阻抗_Ω,鈦陽極接地電阻A1_Ω,鈦陽極接地電阻A2_Ω,鈦陽極接地電阻A3_Ω,鈦陽極接地電阻A4_Ω,鈦陽極總接地電阻_Ω,本次電力_KWH,平均電力值_KWH/日,交流輸入_A,交流輸入_V,直流輸出_A,直流輸出_V,季節
6,24,111Q1,2022-03-17 00:00:00,-1473,-1287,2.56,2.27,2.23,2.26,3.45,2.03,7659,8.142857,0.70,233.6,1.48,3.731,1
7,24,111Q2,2022-05-27 00:00:00,-1429,-1272,2.52,2.44,2.42,2.41,3.84,2.28,8178,7.309859,0.73,232.0,1.72,3.726,2
8,24,111Q3,2022-08-03 00:00:00,-1467,-1287,0.74,3.52,3.58,3.60,5.12,3.43,8684,7.441176,0.71,232.4,1.75,3.741,3
9,24,111Q4,2022-11-03 00:00:00,-1498,-1360,0.57,3.48,3.37,3.56,5.20,3.63,9366,7.413043,0.71,224.8,1.78,1.817,4
16,25,111Q1,2022-03-17 00:00:00,-1390,-1321,2.73,1.58,1.47,1.35,1.42,1.12,22270,2.993506,0.41,233.8,0.49,0.860,1
17,25,111Q2,2022-05-25 00:00:00,-1340,-1321,2.32,2.02,1.98,1.87,1.92,1.75,22496,3.275362,0.41,228.3,0.61,0.945,2
18,25,111Q3,2022-08-03 00:00:00,-1364,-1201,0.45,1.01,0.97,0.95,0.97,0.87,22724,3.257143,0.67,230.6,2.43,1.500,3
19,25,111Q4,2022-11-02 00:00:00,-1408,-1360,3.94,0.97,0.96,0.98,0.95,0.83,23054,3.626374,0.59,232.6,1.38,1.443,4


In [8]:
df = pd.merge(df, corrosion, on = "季節")

In [9]:
print(df.shape)
df.to_csv("data/parser/rectifier_corrosion.csv", index = False)

(80, 25)
